In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, UnexpectedAlertPresentException
import time
import random
from selenium.webdriver.chrome.service import Service


In [2]:
# --- Configuración ---
# Reemplaza con la ruta real a tu archivo Excel
EXCEL_PATH = 'Actualización Censo Resguardo Indigena de Guachucal - 2025 (1).xlsx'
# Reemplaza con el nombre de la hoja si no es la primera
SHEET_NAME = 'Hoja1'    
# Reemplaza con los nombres exactos de tus columnas
COL_TIPO_DOC = 'TIPO IDENTIFICACION'
COL_NUM_DOC = 'NUMERO DOCUMENTO'
# Nueva columna para guardar el resultado
COL_RESULTADO = 'Estado_Sisben'
# Reemplaza con la URL real de la página de consulta del Sisbén
URL_SISBEN = 'https://reportes.sisben.gov.co/dnp_sisbenconsulta' # ¡Importante! Debes poner la URL correcta aquí
# Ruta al ejecutable del WebDriver que descargaste
WEBDRIVER_PATH = r'C:\chromedriver.exe' # O geckodriver, etc.

# Tiempos de espera (en segundos)
MIN_DELAY = 1  # Pausa mínima entre consultas
MAX_DELAY = 4 # Pausa máxima entre consultas
WAIT_TIMEOUT = 4 # Tiempo máximo de espera para que aparezcan elementos

# --- Mapeo de Tipos de Documento (EJEMPLO - AJUSTAR SEGÚN LA WEB) ---
# Debes averiguar los valores exactos que usa el <select> en la web
# El formato es: { 'Texto en tu Excel': 'valor_del_option_en_HTML' }
MAPEO_TIPO_DOC = {
    'RC': '1', # Cédula de Ciudadanía (Ejemplo)
    'TI': '2', # Cédula de Extranjería (Ejemplo)
    'CC': '3', # Tarjeta de Identidad (Ejemplo)
    'CE': '4', # Registro Civil (Ejemplo)
    # Agrega más tipos si es necesario
}

# --- Funciones Auxiliares ---
def configurar_driver():
    """Configura e inicia el WebDriver."""
    # Opciones para el navegador (puedes añadir más)
    options = webdriver.ChromeOptions()
    #options.add_argument('--headless')  # Ejecutar sin abrir ventana de navegador (puede ser detectado)
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    try:
        driver = webdriver.Chrome(executable_path=WEBDRIVER_PATH, options=options)
        return driver
    except Exception as e:
        print(f"Error al iniciar WebDriver: {e}")
        print(f"Asegúrate de que la ruta '{WEBDRIVER_PATH}' es correcta y el WebDriver es compatible con tu navegador.")
        return None

def consultar_sisben(driver, tipo_doc_excel, num_doc):
    """Realiza una única consulta en la página del Sisbén."""
    try:
        driver.get(URL_SISBEN)
        wait = WebDriverWait(driver, WAIT_TIMEOUT)

        # --- ¡IMPORTANTE! AJUSTA LOS LOCALIZADORES (By.ID, By.NAME, etc.) ---
        # Estos son ejemplos, DEBES encontrarlos inspeccionando la página real

        # 1. Seleccionar Tipo de Documento
        try:
            tipo_doc_select_element = wait.until(EC.presence_of_element_located((By.ID, 'TipoID'))) # Reemplaza 'id_del_dropdown_tipo_doc'
            select = Select(tipo_doc_select_element)
            valor_html = MAPEO_TIPO_DOC.get(str(tipo_doc_excel).strip().upper())
            if not valor_html:
                print(f"Advertencia: Tipo de documento '{tipo_doc_excel}' no mapeado. Saltando.")
                return "Error: Tipo Doc No Mapeado"
            select.select_by_value(valor_html) # O select_by_visible_text si prefieres
        except (NoSuchElementException, TimeoutException):
            print("Error: No se encontró el dropdown de tipo de documento.")
            return "Error: Elemento no encontrado (Tipo Doc)"
        except Exception as e:
            print(f"Error seleccionando tipo de documento: {e}")
            return "Error: Selección Tipo Doc"


        # 2. Ingresar Número de Documento
        try:
            num_doc_input = wait.until(EC.presence_of_element_located((By.ID, 'documento'))) # Reemplaza 'id_del_input_numero_doc'
            num_doc_input.clear()
            num_doc_input.send_keys(str(num_doc).strip())
        except (NoSuchElementException, TimeoutException):
            print("Error: No se encontró el campo de número de documento.")
            return "Error: Elemento no encontrado (Num Doc)"
        except Exception as e:
            print(f"Error ingresando número de documento: {e}")
            return "Error: Input Num Doc"


        # 3. Hacer clic en Consultar
        try:
            # Puede haber alertas inesperadas ANTES de hacer clic si llenaste mal algo
            boton_consultar = wait.until(EC.element_to_be_clickable((By.ID, 'botonenvio'))) # Reemplaza 'id_del_boton_consultar'
            boton_consultar.click()
        except UnexpectedAlertPresentException:
            try:
                alert = driver.switch_to.alert
                alert_text = alert.text
                print(f"Alerta inesperada ANTES de consultar: {alert_text}")
                alert.accept()
                return f"Alerta Previa: {alert_text}"
            except Exception as alert_err:
                print(f"Error manejando alerta previa: {alert_err}")
                return "Error: Manejo Alerta Previa"
        except (NoSuchElementException, TimeoutException):
            print("Error: No se encontró el botón de consultar.")
            return "Error: Elemento no encontrado (Botón)"
        except Exception as e:
            print(f"Error haciendo clic en consultar: {e}")
            return "Error: Clic Consultar"

        # 4. Esperar y obtener resultado
        time.sleep(1) # Pequeña pausa para que la página reaccione

        resultado = "No Determinado"
        try:
            # Intenta manejar una alerta (ventana emergente)
            alert = wait.until(EC.alert_is_present())
            alert_text = alert.text
            if "El tipo de identificación" in alert_text.lower() or "NO se encuentra en la base" in alert_text.lower(): # Ajusta según el texto real
                resultado = "No tiene Sisben (Alerta)"
            else:
                resultado = f"Alerta: {alert_text}" # Podría ser otro mensaje
            alert.accept()

        except TimeoutException:
            # Si no hay alerta, busca el resultado en la página
            persona_con_sisben = False
            elementos_exito = [
                (By.XPATH, '/html/body/div[1]/main/div/div[1]/div[3]/div/div/div[2]/div[1]/p[1]'), # Ejemplo XPATH (puede que necesites varios)
                # Agrega aquí más XPaths para la imagen, nombres o apellidos
                (By.XPATH, '/html/body/div[1]/main/div/div[1]/div[1]/img'),
                (By.XPATH, '/html/body/div[1]/main/div/div[1]/div[3]/div/div/div[2]/div[1]/p[1]'),
                (By.XPATH, '/html/body/div[1]/main/div/div[1]/div[3]/div/div/div[2]/div[2]/p[1]')
            ]

            for by, path in elementos_exito:
                try:
                    wait.until(EC.presence_of_element_located((by, path)))
                    persona_con_sisben = True
                    break # Si encontramos al menos uno, asumimos que tiene Sisbén
                except TimeoutException:
                    pass # Si no se encuentra este elemento, probamos el siguiente

            if persona_con_sisben:
                resultado = "Registrado, persona con sisben"
            else:
                # Si no se encuentra ningún elemento de éxito, busca un mensaje de no encontrado en la página
                try:
                    # ¡DEBES AJUSTAR ESTE LOCALIZADOR!
                    mensaje_no_encontrado = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="swal2-content"]')))# Ejemplo XPATH
                    if mensaje_no_encontrado:
                        resultado = "No Registrado , no tiene sisben"
                except TimeoutException:
                    print("No se pudo determinar el resultado (ni alerta, ni elemento de éxito/fracaso encontrado en tiempo).")
                    resultado = "Error: Resultado Indeterminado"
                except Exception as e:
                    print(f"Error buscando mensaje de no encontrado en la página: {e}")
                    resultado = "Error: Buscando Mensaje No Encontrado"

        except Exception as e:
            print(f"Error general al obtener resultado: {e}")
            resultado = f"Error: {e}"

        return resultado

    except UnexpectedAlertPresentException:
        try:
            alert = driver.switch_to.alert
            alert_text = alert.text
            print(f"Alerta inesperada durante la consulta: {alert_text}")
            alert.accept()
            return f"Error: Alerta Inesperada ({alert_text})"
        except Exception as alert_err:
            print(f"Error manejando alerta inesperada: {alert_err}")
            return "Error: Manejo Alerta Inesperada"
    except Exception as e:
        print(f"Error crítico en la consulta para {num_doc}: {e}")
        # Intenta refrescar o navegar de nuevo en caso de error grave
        try:
            driver.refresh()
        except:
            pass # Ignora si no se puede refrescar
        return "Error Crítico"


# --- Script Principal ---
if __name__ == "__main__":
    print("Iniciando proceso de consulta Sisbén...")

    # Cargar datos de Excel
    try:
        df = pd.read_excel(EXCEL_PATH, sheet_name=SHEET_NAME,header=6)
        print(f"Archivo Excel '{EXCEL_PATH}' cargado. {len(df)} registros encontrados.")
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo Excel en '{EXCEL_PATH}'")
        exit()
    except Exception as e:
        print(f"Error al leer el archivo Excel: {e}")
        exit()

    # Añadir columna de resultados si no existe
    if COL_RESULTADO not in df.columns:
        df[COL_RESULTADO] = ""

    # Iniciar WebDriver
    driver = configurar_driver()
    if not driver:
        exit()

    print("WebDriver iniciado.")

    # *** MODIFICACIÓN PARA INICIAR DESDE LA FILA 2890 ***
    start_row = 3540
    # Recuerda que los índices en pandas empiezan desde 0, y ya has leído el encabezado (6 filas).
    # Asumimos que quieres empezar desde la fila 2890 de los DATOS después del encabezado.
    # Esto correspondería al índice 2889 en el DataFrame.
    df_subset = df.iloc[start_row - 1:] # El slicing es exclusivo del final, así que usamos start_row - 1

    # Iterar sobre las filas del DataFrame a partir de la fila especificada
    for index, row in df_subset.iterrows():
        # El resto de tu código del bucle for se mantiene igual
        # Opcional: Saltar filas ya procesadas (si tienen un valor en COL_RESULTADO)
        if pd.notna(row[COL_RESULTADO]) and row[COL_RESULTADO] != "":
            print(f"Registro (original index {df.index[index] + 1}) ya procesado. Saltando.")
            continue

        tipo_doc = row[COL_TIPO_DOC]
        num_doc = row[COL_NUM_DOC]

        # Validar datos básicos
        if pd.isna(tipo_doc) or pd.isna(num_doc):
            print(f"Registro (original index {df.index[index] + 1}): Datos incompletos (Tipo={tipo_doc}, Num={num_doc}). Saltando.")
            df.loc[index, COL_RESULTADO] = "Error: Datos Incompletos"
            continue

        print(f"Consultando registro (original index {df.index[index] + 1}): Tipo={tipo_doc}, Num={num_doc}")

        # Realizar la consulta
        resultado_consulta = consultar_sisben(driver, tipo_doc, num_doc)
        df.loc[index, COL_RESULTADO] = resultado_consulta
        print(f"Resultado: {resultado_consulta}")

        # Guardar progreso cada X iteraciones
        if (index + 1) % 30 == 0:
            try:
                output_filename = f"{EXCEL_PATH.replace('.xlsx', '')}_progreso_{df.index[index]+1}.xlsx" # Guarda con el índice original
                df.to_excel(output_filename, index=False)
                print(f"Progreso guardado en '{output_filename}'")
            except Exception as e:
                print(f"Error al guardar el progreso: {e}")

        # Pausa aleatoria para evitar bloqueos
        delay = random.uniform(MIN_DELAY, MAX_DELAY)
        print(f"Esperando {delay:.2f} segundos...")
        time.sleep(delay)

    # Cerrar el navegador
    print("Cerrando el navegador...")
    driver.quit()

    # Guardar el resultado final
    try:
        output_filename_final = f"{EXCEL_PATH.replace('.xlsx', '')}_RESULTADOS_FINAL.xlsx"
        df.to_excel(output_filename_final, index=False)
        print(f"¡Proceso completado! Resultados guardados en '{output_filename_final}'")
    except Exception as e:
        print(f"Error al guardar el archivo final: {e}")

    print("Fin del script.")

Iniciando proceso de consulta Sisbén...
Archivo Excel 'Actualización Censo Resguardo Indigena de Guachucal - 2025 (1).xlsx' cargado. 8096 registros encontrados.


C:\Users\jedam\AppData\Local\Temp\ipykernel_17372\3176122859.py:42: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=WEBDRIVER_PATH, options=options)


WebDriver iniciado.
Consultando registro (original index 3540): Tipo=CC, Num=27211643
No se pudo determinar el resultado (ni alerta, ni elemento de éxito/fracaso encontrado en tiempo).
Resultado: Error: Resultado Indeterminado
Error al guardar el progreso: [Errno 13] Permission denied: 'Actualización Censo Resguardo Indigena de Guachucal - 2025 (1)_progreso_3540.xlsx'
Esperando 1.99 segundos...
Consultando registro (original index 3541): Tipo=CC, Num=1088654270
Resultado: No Registrado , no tiene sisben
Esperando 2.68 segundos...
Consultando registro (original index 3542): Tipo=CC, Num=1088653608
Resultado: No Registrado , no tiene sisben
Esperando 2.69 segundos...
Consultando registro (original index 3543): Tipo=CC, Num=27215470
Resultado: No Registrado , no tiene sisben
Esperando 2.18 segundos...
Consultando registro (original index 3544): Tipo=CC, Num=1192774527
Resultado: No Registrado , no tiene sisben
Esperando 3.71 segundos...
Consultando registro (original index 3545): Tipo=CC,